In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# load .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [18]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [22]:
# Import writer module
from source import writer
from source import utils
from source import evaluator

In [20]:
career_story_path = os.path.join(os.getcwd(), '..', 'data', 'career_story.txt')
career_story = utils.load_career_story(career_story_path)

jd_path = os.path.join(os.getcwd(), '..', 'data', 'clio_jd.txt')
jd = utils.load_job_description(jd_path)

In [43]:
customized_criteria = evaluator.gen_customized_cv_criteria(jd=jd, model='o1')

In [45]:

cv_v1 = writer.generate_cv(career_story=career_story, jd=jd, customized_criteria=customized_criteria, model='o1')

In [51]:
missing_criteria = evaluator.check_missing_criteria(customized_cv_criteria=customized_criteria, cv_string=cv_v1, model='gpt-4o-mini')

In [54]:
print(customized_criteria)

CV SUCCESS CRITERIA

[MUST-HAVE CRITERIA]
1. Minimum 5+ Years in Data Science: Demonstrate at least five years of hands-on, applied data science experience (as specified in the JD). Include examples of building models, analyzing large datasets, and successfully delivering data solutions.  
2. Product and User Behavior Analytics: Highlight experience with metrics related to product usage, retention, churn analysis, and other user behavior insights—explicitly reference projects involving data-driven product decisions.  
3. Proficiency in Python, SQL, and ML Frameworks: Show advanced skill in Python-based data pipelines (pandas, PyTorch, scikit-learn). Emphasize SQL expertise, database modeling, and data warehousing principles.  
4. End-to-End ML/AI Pipeline Deployment: Point out hands-on experience deploying machine learning or AI models to production, specifically mentioning tools like MLflow or Kedro if possible.  
5. Strong Cross-Functional Collaboration & Communication: Provide clear

In [53]:
print(missing_criteria)

{
  "missing_criteria": [
    "Criterion 1 is not met because while the CV states 5+ years of experience, it lacks specific examples of building models or analyzing large datasets that successfully delivered data solutions.",
    "Criterion 2 is not fully met; although the CV mentions churn analysis and product growth, it lacks clear references to specific projects that involved data-driven product decisions shaped by user behavior insights.",
    "Criterion 4 is not fully satisfied; while the CV mentions deploying Machine Learning models, there is no explicit reference to using tools like MLflow or Kedro for deployment.",
    "Criterion 6 is not met because the CV does not provide specific examples of projects where strategic or growth-focused needs were translated into predictive or prescriptive models."
  ]
}


In [ ]:
cv_v2 = evaluator.check_truthfulness(cv_string=cv_v1, real_info=career_story, model='gpt-4o-mini')

In [39]:
cv_path = os.path.join(os.getcwd(), '..', 'output', 'cv.html')

In [40]:
utils.string_to_html_file(html_string=truthful_cv, output_filepath=cv_path)

2025-03-21 16:52:14.326 | SUCCESS  | source.utils:string_to_html_file:12 - HTML file created successfully at: /Users/phamanhdung/Documents/genius-cv/notebook/../output/cv.html


In [41]:
cv_path = os.path.join(os.getcwd(), '..', 'output', 'draft_cv.html')
utils.string_to_html_file(html_string=draft_cv, output_filepath=cv_path)

2025-03-21 16:55:44.569 | SUCCESS  | source.utils:string_to_html_file:12 - HTML file created successfully at: /Users/phamanhdung/Documents/genius-cv/notebook/../output/draft_cv.html
